# Concise Implementation of Linear Regression
:label:`sec_linear_djl`

Broad and intense interest in deep learning for the past several years
has inspired both companies, academics, and hobbyists
to develop a variety of mature open source frameworks
for automating the repetitive work of implementing
gradient-based learning algorithms.
In the previous section, we relied only on
(i) `NDArray` for data storage and linear algebra;
and (ii) `GradientCollector` for calculating derivatives.
In practice, because data iterators, loss functions, optimizers,
and neural network layers (and some whole architectures)
are so common, modern libraries implement these components for us as well.

In this section, we will show you how to implement
the linear regression model from :numref:`sec_linear_scratch`
concisely by using DJL.

## Generating the Dataset

To start, we will generate the same dataset as in the previous section.

In [ ]:
%mavenRepo snapshots https://oss.sonatype.org/content/repositories/snapshots/

%maven ai.djl:api:0.7.0-SNAPSHOT
%maven ai.djl:model-zoo:0.7.0-SNAPSHOT
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26
%maven net.java.dev.jna:jna:5.3.0
%maven ai.djl.mxnet:mxnet-engine:0.7.0-SNAPSHOT
%maven ai.djl.mxnet:mxnet-native-auto:1.7.0-a

In [ ]:
%load ../utils/DataPoints.java
%load ../utils/Training.java

In [ ]:
import java.nio.file.*;

import ai.djl.*;
import ai.djl.metric.*;
import ai.djl.ndarray.*;
import ai.djl.ndarray.types.*;
import ai.djl.ndarray.index.*;
import ai.djl.nn.*;
import ai.djl.nn.core.*;
import ai.djl.training.*;
import ai.djl.training.initializer.*;
import ai.djl.training.loss.*;
import ai.djl.training.listener.*;
import ai.djl.training.evaluator.*;
import ai.djl.training.optimizer.*;
import ai.djl.training.optimizer.learningrate.*;
import ai.djl.training.dataset.*;
import ai.djl.util.*;

In [ ]:
NDManager manager = NDManager.newBaseManager();

NDArray trueW = manager.create(new float[]{2, -3.4f});
float trueB = 4.2f;

DataPoints dp = DataPoints.syntheticData(manager, trueW, trueB, 1000);
NDArray features = dp.getX();
NDArray labels = dp.getY();

## Reading the Dataset

Just like in the last section,
we can call upon DJL's `dataset` package to read data.
The first step will be to instantiate an `ArrayDataset`.
Here, we set the `features` and `labels` as parameters.
We also specify a `batchSize`
and specify a Boolean value `shuffle` indicating whether or not
we want the `ArrayDataset` to randomly sample the data.

In [ ]:
// Saved in the utils file for later use
public ArrayDataset loadArray(NDArray features, NDArray labels, int batchSize, boolean shuffle) {
    return new ArrayDataset.Builder()
                  .setData(features) // set the features
                  .optLabels(labels) // set the labels
                  .setSampling(batchSize, shuffle) // set the batch size and random sampling
                  .build();
}

int batchSize = 10;
ArrayDataset dataset = loadArray(features, labels, batchSize, false);

To verify that it is working, we can read and print
the first minibatch of instances.

In [ ]:
for (Batch batch : dataset.getData(manager)) {
    NDArray X = batch.getData().head();
    NDArray y = batch.getLabels().head();
    System.out.println(X);
    System.out.println(y);
    batch.close();
    break;
}

## Defining the Model

When we implemented linear regression from scratch
(in :numref:`sec_linear_scratch`),
we defined our model parameters explicitly
and coded up the calculations to produce output
using basic linear algebra operations.
You *should* know how to do this.
But once your models get more complex,
and once you have to do this nearly every day,
you will be glad for the assistance.
The situation is similar to coding up your own blog from scratch.
Doing it once or twice is rewarding and instructive,
but you would be a lousy web developer
if every time you needed a blog you spent a month
reinventing the wheel.

For standard operations, we can use DJL's predefined blocks,
which allow us to focus especially
on the layers used to construct the model
rather than having to focus on the implementation.
To define a linear model, we first import the `Model` class,
which defines a lot of useful methods to interact with our `model`.
We will first define a model variable `model`.
We will then instantiate a SequentialBlock variable `net`, which will refer to an instance of the `SequentialBlock` class. The `SequentialBlock` class defines a container for several layers that will be chained together. Given input data, a `SequentialBlock` passes it through the first layer, in turn passing the output as the second layer’s input and so forth. In the following example, our model consists of only one layer, so we do not really need `SequentialBlock`. But since nearly all of our future models will involve multiple layers, we will use it anyway just to familiarize you with the most standard workflow.

Recall the architecture of a single-layer network as shown in :numref:`fig_singleneuron`.
The layer is said to be *fully-connected*
because each of its inputs are connected to each of its outputs
by means of a matrix-vector multiplication.
In DJL, we can use a `Linear` block to apply a linear transformation.
We simply set the number of outputs (in our case its set to 1) and choose
if we want to include a bias(yes).

![Linear regression is a single-layer neural network. ](https://raw.githubusercontent.com/awslabs/djl-resources/311169c9dbd89e1597a5ae6b0b1ba8a402b8b55e/jupyter/d2l-java/img/singleneuron.svg)
:label:`fig_singleneuron`

In [ ]:
Model model = Model.newInstance("lin-reg");

SequentialBlock net = new SequentialBlock();
Linear linearBlock = Linear.builder().optBias(true).setUnits(1).build();
net.add(linearBlock);

model.setBlock(net);

## Defining the Loss Function

In DJL, the `Loss` class defines various loss functions.
We will use the imported class `Loss`.
In this example, we will use the DJL
implementation of squared loss (`L2Loss`).

In [ ]:
Loss l2loss = Loss.l2Loss();

## Defining the Optimization Algorithm

Minibatch SGD and related variants
are standard tools for optimizing neural networks
and thus DJL supports SGD alongside a number of
variations on this algorithm through its `Optimizer` class.
When we instantiate the `Optimizer`,
we will specify the optimization algorithm we wish to use (`sgd`).
We can also manually set hyper-parameters.
SGD just requires `learningRate`,
here we set it to a fixed rate of 0.03.

In [ ]:
LearningRateTracker lrt = LearningRateTracker.fixedLearningRate(0.03f);
Optimizer sgd = Optimizer.sgd().setLearningRateTracker(lrt).build();

## Instantiate Configuration and Trainer
Now we'll create a training configuration that
describes how we want to train our model.
We will then initialize a `trainer` to do the
training for us.

In [ ]:
DefaultTrainingConfig config = new DefaultTrainingConfig(l2loss)
    .optOptimizer(sgd) // Optimizer (loss function)
    .addTrainingListeners(TrainingListener.Defaults.logging()); // Logging

Trainer trainer = model.newTrainer(config);

## Initializing Model Parameters

Before training our model, we need to initialize the model parameters,
such as the weights and biases in the linear regression model.
We simply call the `initialize` function with the shape of the model
we are training.

In [ ]:
// First axis is batch size - won't impact parameter initialization
// Second axis is the input size
trainer.initialize(new Shape(batchSize, 2));

## Metrics
Normally, DJL doesn't record metrics unless explicitly told to
as recording metrics impacts the execution flow optimizations.
To record metrics, we must instantiate `metrics` from outside
the `trainer` object and then pass it in.

In [ ]:
Metrics metrics = new Metrics();
trainer.setMetrics(metrics);

## Training

You might have noticed that expressing our model through DJL
requires comparatively few lines of code.
We did not have to individually allocate parameters,
define our loss function, or implement stochastic gradient descent.
Once we start working with much more complex models,
DJL's advantages will grow considerably.
However, once we have all the basic pieces in place,
the training loop itself is strikingly similar
to what we did when implementing everything from scratch.

To refresh your memory: for some number of epochs,
we will make a complete pass over the dataset (train_data),
iteratively grabbing one minibatch of inputs
and the corresponding ground-truth labels.
For each minibatch, we go through the following ritual:

* Generate predictions, calculate loss, and calculate gradients by calling `trainBatch(batch)` (forward pass and backward pass).
* Update the model parameters by invoking the `step` function.

`Logging` will automatically print the evaluators we are watching
during each epoch.

In [ ]:
int numEpochs = 3;

for (int epoch = 1; epoch <= numEpochs; epoch++) {
    System.out.printf("Epoch %d\n", epoch);
    // Iterate over dataset
    for (Batch batch : trainer.iterateDataset(dataset)) {
        // Update loss and evaulator
        EasyTrain.trainBatch(trainer, batch);
        
        // Update parameters
        trainer.step();
        
        batch.close();
    }
    // reset training and validation evaluators at end of epoch
    trainer.notifyListeners(listener -> listener.onEpoch(trainer));
}

Below, we compare the model parameters learned by training on finite data
and the actual parameters that generated our dataset.
To access parameters with DJL,
we first access the layer that we need from `model`
and then access that layer's weight and bias through its parameter list
by calling `getParameters`.
We then simply get each param with `get`.
Here, `get(0)` and `get(1)` returns the weights and bias respectively.
As in our from-scratch implementation,
note that our estimated parameters are
close to their ground truth counterparts.

In [ ]:
Block layer = model.getBlock();
ParameterList params = layer.getParameters();
NDArray wParam = params.get(0).getValue().getArray();
NDArray bParam = params.get(1).getValue().getArray();

float[] w = trueW.sub(wParam.reshape(trueW.getShape())).toFloatArray();
System.out.printf("Error in estimating w: [%f %f]\n", w[0], w[1]);
System.out.print("Error in estimating b: ");
System.out.println(trueB - bParam.getFloat());

## Saving Your Model
Now that you have trained your model, you probably want to save it
for future use. Additionally, you probably also want to add
metadata such as training accuracy and epochs trained.
You can do this easily. Simply point to a file location with `Paths.get`.
Metadata can be saved with the `setProperty` method.
Then call the `save` method on the model to save it!

In [ ]:
Path modelDir = Paths.get("../models/lin-reg");
Files.createDirectories(modelDir);

model.setProperty("Epoch", Integer.toString(numEpochs)); // save epochs trained as metadata

model.save(modelDir, "lin-reg");

model

## Summary

* Using DJL, we can implement models much more succinctly.
* In DJL, the `training.dataset` package provides tools for data processing, the `nn` package defines a large number of neural network layers, and the `Loss` class defines many common loss functions.
* DJL's `training.initializer` package provides various methods for model parameter initialization.


## Exercises

1. Review the DJL documentation to see what loss functions and initialization methods are provided in the class `Loss` and `Trainer`. Replace the loss with L1 Loss.
1. How do you access the parameters during training?
